In [14]:
import os
import pprint

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab


### Understanding Semantic Kernel Components

Before diving into the implementation, let's understand the key components of Semantic Kernel.

#### The Kernel

The Kernel is the central orchestrator in Semantic Kernel. It manages:

- **Connecting to AI Models**: Interfaces with various AI models via connectors.
- **Registering and Invoking Plugins**: Manages the lifecycle and execution of plugins.
- **Managing Memory and Context**: Maintains state and context across interactions.
- **Interacting with the Planner**: Coordinates with the Planner to achieve specified goals.

#### Connectors

Connectors allow the Kernel to interface with various AI models and services. They define how the Kernel communicates with these models, whether they're:

- **OpenAI Models**: Such as GPT-3.5, GPT-4.
- **Azure OpenAI Services**: Leveraging Microsoft's cloud-based AI capabilities.
- **Local Models**: Using libraries like Hugging Face Transformers.

#### Plugins

Plugins are modular units that extend the Kernel's functionality. They consist of:

- **Prompt Functions**: Use natural language prompts to interact with AI models.
- **Native Functions**: Written in Python, performing deterministic tasks or interfacing with external services.

Plugins act as building blocks for complex workflows.

#### The Planner

The Planner uses AI to dynamically create a sequence of actions (a plan) to achieve a specified goal. It considers:

- **Available Plugins and Their Functions**: Understands the capabilities of each plugin.
- **Function Descriptions**: Uses metadata to understand what each function does.
- **Combining Functions**: Determines how functions can be combined to fulfill the goal.


# Building Smart Entity and Intent Recognition

 This guide illustrates a robust system with two key skills: **Intent Classification** and **Named Entity Recognition (NER)**. The architecture involves:

### Key Skills

1. **Intent Classification**: Determines the purpose of the user’s query, leveraging LLMs.
2. **Entity Recognition**: Extracts entities like names, dates, or medical terms, using advanced Named Entity Recognition (NER) capabilities also laveraging LLMs.

By leveraging Semantic Kernel's capabilities, we can create a robust system that avoids single points of failure by running in parallel and utilizing multiple skills. Let's dive in!

### Implementation

We will:

1. **Define Plugins**: Create plugins for the Medical Researcher, Clinical Evaluator, and Medical Editor.
2. **Configure the Kernel**: Set up the Kernel with the necessary connectors and plugins.
3. **Develop the Planner**: Implement the Planner to dynamically orchestrate the workflow.
4. **Execute the Plan**: Run the system to produce high-quality medical documentation.

In [15]:
#!pip install semantic-kernel --upgrade
import semantic_kernel as sk
print(f"We are using the following semantic_kernel library version: {sk.__version__}")

We are using the following semantic_kernel library version: 1.17.0


## 1. Define Plugins

We have defined the plugin for MedicalAgents located in the specified directory  `C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store`

## 2. Configure The Kernel

In [16]:
import semantic_kernel as sk

# Initialize the kernel
kernel = sk.Kernel()

In [17]:
# add Azure OpenAI service connector to the kernel
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_CHAT_DEPLOYMENT_ID = os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID_MINI")

service_id = "openai-chat"
# Add Azure OpenAI chat completion
kernel.add_service(AzureChatCompletion(
    service_id=service_id,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_ID,
    api_key=AZURE_OPENAI_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
))

In [18]:
print("Registered services:", kernel.services)

Registered services: {'openai-chat': AzureChatCompletion(ai_model_id='gpt-4o-mini', service_id='openai-chat', client=<openai.lib.azure.AsyncAzureOpenAI object at 0x0000018BBF179E10>, ai_model_type=<OpenAIModelTypes.CHAT: 'chat'>, prompt_tokens=0, completion_tokens=0, total_tokens=0)}


In [19]:
# Define the parent directory and plugin name
parent_directory = os.path.abspath(os.path.join("src", "plugins"))
plugin_name = "plugins_store"

# Add the plugin to the kernel
plugin = kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

print("Available functions in intentPlugin plugin:")

print("Loaded plugin functions:", plugin.functions.keys())

print(plugin.functions['intentPlugin'].metadata)


[2024-12-19 08:08:48 - semantic_kernel.functions.kernel_plugin:332 - WARNING] Failed to create function from directory: C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store\medResearch
[2024-12-19 08:08:48 - semantic_kernel.functions.kernel_plugin:332 - WARNING] Failed to create function from directory: C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store\nlpToSqlPlugin
[2024-12-19 08:08:48 - semantic_kernel.functions.kernel_plugin:332 - WARNING] Failed to create function from directory: C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store\QueryDb


Available functions in intentPlugin plugin:
Loaded plugin functions: dict_keys(['intentPlugin', 'nerPlugin'])
name='intentPlugin' plugin_name='plugins_store' description='Classify a sports-related user query into one of the predefined intent categories (e.g., summarize, historical, scores, etc.)' parameters=[KernelParameterMetadata(name='input', description='The user’s sports-related query or statement to be classified', default_value='', type_='', is_required=True, type_object=None, schema_data={'type': 'object', 'description': 'The user’s sports-related query or statement to be classified'}, include_in_function_choices=True)] is_prompt=True is_asynchronous=True return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None, schema_data=None, include_in_function_choices=True) additional_properties=None


In [22]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

arguments = KernelArguments(input="I just need a high-level recap of the Warriors’ playoff performance.")
print("Arguments:", arguments)

result = await kernel.invoke(plugin.functions['intentPlugin'], arguments, service_id="openai-chat")
print("Result:", result)

Arguments: {'input': 'I just need a high-level recap of the Warriors’ playoff performance.'}
Result: ```json
{ "intent": "summarize" }
```


In [23]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

arguments = KernelArguments(input="I just need a high-level recap of the Warriors’ playoff performance.")
print("Arguments:", arguments)

result = await kernel.invoke(plugin.functions['nerPlugin'], arguments, service_id="openai-chat")
print("Result:", result)

Arguments: {'input': 'I just need a high-level recap of the Warriors’ playoff performance.'}
Result: ```json
{
  "entities": [
    { "text": "Warriors", "category": "Organization" },
    { "text": "playoff", "category": "Event" }
  ]
}
```


## 3. Develop the Planer

In [24]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto() # the automatic planning loop https://learn.microsoft.com/en-us/semantic-kernel/concepts/planning?pivots=programming-language-python#the-automatic-planning-loop


## 4. Execute the Plan

In [25]:
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
import logging
from semantic_kernel.utils.logging import setup_logging

# Create a history of the conversation
chat_history = ChatHistory()

# Add system message to the chat history to define the system's role
chat_history.add_system_message(
    "You are an AI-powered assistant specializing in Named Entity Recognition (NER) and Intent Classification. "
    "Your goal is to analyze user input, extract named entities, classify the user's intent, and return results in a structured JSON format. "
    "The JSON should include the following fields: "
    "1. 'entities': A list of named entities found in the user input, each with 'text' and 'category' fields. "
    "2. 'intent': The classified intent of the user input. "

    "The intent can only be one of the following categories: "
    "- summarize "
    "- historical "
    "- statistics "
    "- schedule "
    "- roster "
    "- standings "
    "- support "
    "- catchMeUp "
    "- scores "
    "- None "

    "The named entities should be classified into one of the following categories: "
    "1. **Person**: Names of individuals. "
    "2. **PersonType**: Roles or job titles (e.g., 'coach', 'player'). "
    "3. **Location**: Geographic or geopolitical entities (e.g., 'Paris', 'Dodger Stadium'). "
    "4. **Organization**: Teams, companies, or institutions (e.g., 'ESPN', 'New York Yankees'). "
    "5. **Event**: Sporting events, historical events, or cultural holidays (e.g., 'Super Bowl 2023'). "
    "6. **Product**: Physical or conceptual items (e.g., 'iPhone 15'). "
    "7. **Skill**: Capabilities or expertise (e.g., '3-point shooting', 'machine learning'). "
    "8. **Address**: Mailing or physical addresses (e.g., '1600 Pennsylvania Avenue'). "
    "9. **PhoneNumber**: Telephone numbers in any supported format (e.g., '+1-202-555-0191'). "
    "10. **Email**: Email addresses (e.g., 'contact@example.com'). "
    "11. **URL**: Links to websites (e.g., 'https://example.com'). "
    "12. **IP**: Network IP addresses (e.g., '192.168.0.1'). "
    "13. **DateTime**: Dates, times, or time-related phrases (e.g., 'next Monday', 'July 4, 2025'). "
    "14. **Quantity**: Numerical values, measurements, or percentages (e.g., '50%', '6 feet tall'). "

    "Here are some examples of the expected JSON format: "

    "Example 1: "
    "{ "
    "'entities': ["
    "{ 'text': 'LeBron James', 'category': 'Person' }, "
    "{ 'text': '50', 'category': 'Quantity' }, "
    "{ 'text': 'yesterday', 'category': 'DateTime' }, "
    "{ 'text': 'Warriors', 'category': 'Organization' }"
    "], "
    "'intent': 'scores' "
    "}, "

    "Example 2: "
    "{ "
    "'entities': ["
    "{ 'text': 'Paris', 'category': 'Location' }, "
    "{ 'text': 'Eiffel Tower', 'category': 'Location' }, "
    "{ 'text': 'next week', 'category': 'DateTime' }"
    "], "
    "'intent': 'schedule' "
    "}, "

    "Example 3: "
    "{ "
    "'entities': ["
    "{ 'text': 'Apple', 'category': 'Organization' }, "
    "{ 'text': 'iPhone 15', 'category': 'Product' }, "
    "{ 'text': 'release date', 'category': 'DateTime' }"
    "], "
    "'intent': 'None' "
    "}"
)

# Add user message to the chat history
user_input = "I just need a high-level recap of the Warriors’ playoff performance."
chat_history.add_user_message(user_input)

# Set up logging
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
setup_logging()
logging.getLogger("kernel").setLevel(logging.INFO)

# Get the response from the AI
result = (await chat_completion.get_chat_message_contents(
    chat_history=chat_history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(input="The impact of AI on medical diagnostics"),
))[0]

# Print the result
print("AI Response:", result)

AI Response: ```json
{
  "entities": [
    {
      "text": "Warriors",
      "category": "Organization"
    },
    {
      "text": "playoff",
      "category": "Event"
    }
  ],
  "intent": "summarize"
}
```
